In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [30]:
dem_df = pd.read_csv('../data/post_dem_candidates_with_tweet_topics.csv', encoding="ISO-8859-1").dropna(subset=['primary_pctg'])
rep_df = pd.read_csv('../data/post_rep_candidates_with_tweet_topics.csv', encoding="ISO-8859-1").dropna(subset=['primary_pctg'])
tweets_with_topics = pd.read_csv('../data/tweet_data/tweets_with_topics.csv', index_col=False)
# sa_all_tweets = pd.read_csv('../data/tweet_data/sa_all_tweets_post.csv', index_col=False)

all_tweets_cleaned = pd.read_csv('../data/tweet_data/all_tweets_cleaned.csv', index_col=False)


In [39]:
tweets_with_topics = tweets_with_topics.merge(all_tweets_cleaned[['tweet_idx', 'twitter_handle']], how='left', on=['tweet_idx'])
                                              
                                              
                                              

In [34]:
print(len(all_tweets_cleaned))
print(len(tweets_with_topics))

557549
557549


In [43]:
dem_candidates = dem_df.twitter_handle.unique()
dem_candidates2 = dem_df.twitter_handle2.unique()

party = []
for i, r in tweets_with_topics.iterrows():
    if i % 20000 == 0:
        print('{} / {}'.format(i, len(tweets_with_topics)))
    if r['twitter_handle'] in dem_candidates or r['twitter_handle'] in dem_candidates2:
        party.append(0)
    else:
        party.append(1)
tweets_with_topics['party'] = party

0 / 557549
20000 / 557549
40000 / 557549
60000 / 557549
80000 / 557549
100000 / 557549
120000 / 557549
140000 / 557549
160000 / 557549
180000 / 557549
200000 / 557549
220000 / 557549
240000 / 557549
260000 / 557549
280000 / 557549
300000 / 557549
320000 / 557549
340000 / 557549
360000 / 557549
380000 / 557549
400000 / 557549
420000 / 557549
440000 / 557549
460000 / 557549
480000 / 557549
500000 / 557549
520000 / 557549
540000 / 557549


In [44]:
topics = ['gun_control', 'health_care',
       'abortion', 'immigration', 'education', 'jobs'
          , 'environment','women', 'lgbt', 'freedom', 'trump']
topic_cols = ['{}_tweet_count'.format(t) for t in topics]

In [45]:
print("dem candidates w/ no classified topics: ", len(dem_df[(dem_df['gun_control_tweet_count'] == 0) & 
          (dem_df['health_care_tweet_count'] == 0) &
          (dem_df['abortion_tweet_count'] == 0) &
          (dem_df['immigration_tweet_count'] == 0) &
          (dem_df['education_tweet_count'] == 0) &
          (dem_df['jobs_tweet_count'] == 0) &
          (dem_df['environment_tweet_count'] == 0) &
          (dem_df['women_tweet_count'] == 0) &
          (dem_df['lgbt_tweet_count'] == 0) &
          (dem_df['freedom_tweet_count'] == 0) & (dem_df['trump_tweet_count'] == 0)]) / len(dem_df))

print("rep candidates w/ no classified topics: ", len(rep_df[(rep_df['gun_control_tweet_count'] == 0) & 
          (rep_df['health_care_tweet_count'] == 0) &
          (rep_df['abortion_tweet_count'] == 0) &
          (rep_df['immigration_tweet_count'] == 0) &
          (rep_df['education_tweet_count'] == 0) &
          (rep_df['jobs_tweet_count'] == 0) &
          (rep_df['environment_tweet_count'] == 0) &
          (rep_df['women_tweet_count'] == 0) &
          (rep_df['lgbt_tweet_count'] == 0) &
          (rep_df['freedom_tweet_count'] == 0) & (rep_df['trump_tweet_count'] == 0)]) / len(rep_df))

dem candidates w/ no classified topics:  0.22685788787483702
rep candidates w/ no classified topics:  0.4287769784172662


In [46]:


def get_topic_df(candidate_df, party):
    max_count_col = []
    median_count_col = []
    avg_count_col = []
    total_classified_col = []
    
    for t in topics:
        topic_count = len(tweets_with_topics[(tweets_with_topics[t] == 1) & (tweets_with_topics['party'] == party)])
        max_count = candidate_df["{}_tweet_count".format(t)].max()
        avg_count = candidate_df["{}_tweet_count".format(t)].mean()
        max_count_col.append(max_count)
        avg_count_col.append(round(avg_count, 2))
        total_classified_col.append(topic_count)
    df = pd.DataFrame(np.transpose([topics,avg_count_col, max_count_col, total_classified_col])
             , columns=['topic', 'avg_count', 'max_count', 'total_classified'])
    df['max_count'] = df['max_count'].astype('int32')
    df['total_classified'] = df['total_classified'].astype('int32')
    return df
    
dem_topic_df = get_topic_df(dem_df, 0)
rep_topic_df = get_topic_df(rep_df, 1)


In [60]:
print("total tweets: ",len(tweets_with_topics[tweets_with_topics['party'] == 0]))
dem_topic_df.sort_values(by=['total_classified'], ascending=False).reset_index(drop=True)

total tweets:  381881


,topic,avg_count,max_count,total_classified
0,trump,28.24,554,21582
1,health_care,23.49,343,17831
2,gun_control,15.01,352,11455
3,jobs,13.25,157,10071
4,education,9.55,192,7292
5,environment,7.65,1026,5855
6,immigration,7.04,192,5379
7,lgbt,2.39,64,1826
8,freedom,2.16,55,1651
9,women,2.03,59,1548


In [61]:
print("total tweets: ", len(tweets_with_topics[tweets_with_topics['party'] == 1]))
rep_topic_df.sort_values(by=['total_classified'], ascending=False).reset_index(drop=True)

total tweets:  175668


,topic,avg_count,max_count,total_classified
0,trump,19.53,1015,13818
1,immigration,6.15,324,4339
2,jobs,5.74,200,4124
3,gun_control,4.23,254,3086
4,health_care,3.77,473,2956
5,freedom,3.48,276,2418
6,education,2.47,83,1738
7,abortion,1.96,75,1405
8,environment,0.56,33,435
9,women,0.21,10,157
